In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter

In [2]:
train_set = torchvision.datasets.FashionMNIST(root = r"C:\Users\Administrator\Desktop\机器学习，深度学习参考教材\deeplizard--pytorch神经网络入门\准备数据\data_FashionMNIST",
                                              train= True,download = True,transform = transforms.Compose([transforms.ToTensor()]))





torch.set_grad_enabled(True)

In [3]:
class Network(nn.Module):

    def __init__(self):
        super(Network,self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 1,out_channels=6,kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels = 12,kernel_size  =5)



        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.fc3 = nn.Linear(in_features=60,out_features=10)


    def forward(self,t):

        t = self.conv1(t)
        t = F.relu(t)
        t  = F.max_pool2d(t,kernel_size= 2,stride = 2)


        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size = 2,stride = 2)

        t = self.fc1(t.reshape(-1,12*4*4))
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.fc3(t)

        return t

In [4]:
# batch_size = 100
# lr = 0.01

def get_num_correct(preds,labels):
    return  (preds.argmax(dim = 1) == labels).sum()

from itertools import product

parameters = dict(lr = [0.01,0.001],batch_size = [10,100,1000],shuffle = [True,False])
parameters_values = [v for v in parameters.values()]


# network = Network()

# updater = optim.SGD(network.parameters(),lr = lr)
# # loss = F.cross_entropy(preds,labels)
# train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size)




# images,labels = next(iter(train_loader))
# grid = torchvision.utils.make_grid(images)

# comment = f"batch_size = {batch_size},lr= {lr}"
# tb = SummaryWriter("logs",comment=comment)
# tb.add_image("images",grid)
# tb.add_graph(network,images)
for lr,batch_size,shuffle in product(*parameters_values):
    network = Network()
    comment = f"batch_size = {batch_size},lr= {lr},shuffle = {shuffle}"
    # comment = "batch_size =100,lr= 0.01"
    tb = SummaryWriter(comment=comment)
    updater = optim.SGD(network.parameters(),lr = lr)
    # loss = F.cross_entropy(preds,labels)
    train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size)
    for epoch  in range(5):
        train_loss = 0
        train_num_correct = 0
        for images,labels in train_loader:

            preds = network(images)
            train_num_correct += get_num_correct(preds,labels)

            loss = F.cross_entropy(preds,labels)
            train_loss += loss*batch_size

            updater.zero_grad()
            loss.backward()


            updater.step()
        print("lr = ",lr,"batch_size = ",batch_size,
              "epoch=  ",epoch,"train_loss = ",train_loss,"correct_% = ",train_num_correct/len(train_set))

        tb.add_scalar("train_loss",train_loss,epoch)
        tb.add_scalar("train_correct_%",train_num_correct/len(train_set),epoch)
    tb.close()
        # tb.add_histogram("conv1_bias",network.conv1.bias,epoch)
        # tb.add_histogram("conv1_weight",network.conv1.weight,epoch)
        # tb.add_histogram("conv1_weight_grad",network.conv1.weight.grad,epoch)





lr =  0.01 batch_size =  10 epoch=   0 train_loss =  tensor(52995.4609, grad_fn=<AddBackward0>) correct_% =  tensor(0.6629)
lr =  0.01 batch_size =  10 epoch=   1 train_loss =  tensor(30918.0801, grad_fn=<AddBackward0>) correct_% =  tensor(0.8078)
lr =  0.01 batch_size =  10 epoch=   2 train_loss =  tensor(26382.5586, grad_fn=<AddBackward0>) correct_% =  tensor(0.8391)
lr =  0.01 batch_size =  10 epoch=   3 train_loss =  tensor(23659.4609, grad_fn=<AddBackward0>) correct_% =  tensor(0.8554)
lr =  0.01 batch_size =  10 epoch=   4 train_loss =  tensor(21928.9590, grad_fn=<AddBackward0>) correct_% =  tensor(0.8662)
lr =  0.01 batch_size =  10 epoch=   0 train_loss =  tensor(57404.7852, grad_fn=<AddBackward0>) correct_% =  tensor(0.6449)
lr =  0.01 batch_size =  10 epoch=   1 train_loss =  tensor(32099.4434, grad_fn=<AddBackward0>) correct_% =  tensor(0.7999)
lr =  0.01 batch_size =  10 epoch=   2 train_loss =  tensor(27659.4473, grad_fn=<AddBackward0>) correct_% =  tensor(0.8296)
lr =  0.